### 2. Parsing and creating the list of dataframes (call it 'tables')

In [ ]:
# Creating the list of dataframes, called "tables"

# Import packages
import requests
import urllib
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

with open('YOUR_DATA_FILE.csv') as fp:
    def14aLinks = [filename.strip() for filename in fp.readlines()]
    def14aLinks = def14aLinks[1:]
    
tables = []
for link in def14aLinks:
    
    content = requests.get(link)
    soup = BeautifulSoup(content.text, 'lxml')

    try:
        search_result = pd.read_html(link, match="Beneficial", header=0)
        for df in search_result:
            df = df.astype('str')
            for column in df.columns:
                if df[column].str.lower().str.contains("%").any():
                    tables.append(df)
                    break
    except ValueError:
        continue

    # Find and add company name
    try:
        company = soup.select_one("p:contains(Name)").find_previous('p').text.strip()
        df['Company Name'] = company
    except AttributeError:
        pass

    # 5. Adding Date
    try:
        date = soup.find('p', text = re.compile(r"(?:January|February|March|April|May|June|July|August|September|October|November|December)")).text.strip()
        df['Filing Date'] = date
    except AttributeError:
        pass
    #print(df)
    
    #NOTE: THIS IS THE LONGEST PART OF THE CODE. MAY TAKE UP TO 4 HOURS, DEPENDING ON NUMBER OF ITERATIONS

## 3. Saving each dataframe in "tables" in an individual csv file

In [66]:
# Function to save each table to individual csv file
def individual_saver(t, year):
    #index count
    i = 0
    while True:
        try:
            temp_var = t[i]
            temp_var.to_csv('YOUR_PATH_HERE/Raw_csv/{}/sample{}.csv'.format(year, i))
            i += 1
        except IndexError:
            break

individual_saver(tables, 1998)

### 4. Tidying every dataframes in "tables" and appending result to initially created csv (See "Def 14A Final _ First CSV" notebook)

In [67]:
# Tidying and appending every subsequent dataframe to initial CSV
def tidy_saver(t):
    #index count
    i = 0
    while True:
        try:
            temp_var = t[i]
            temp_var = pd.DataFrame(temp_var)
            temp_var.columns = temp_var.iloc[0]
            temp_var = temp_var.drop(temp_var.index[0])
            try:
                temp_var = temp_var.drop(['nan'], axis = 1)
            except KeyError:
                pass
            column_names = ['Beneficial_Owner', 'share','Shares voting power', 'percent','Percent shares', 'Company name', 'Date filed']
            if len(temp_var.columns) == 7:
                temp_var.columns = column_names
                #temp_var = temp_var.drop(['share','percent'], axis = 1) #dropping these may not be ideal
                # as some values could land there. So, just clean them manually later on
            else:
                pass
            temp_var.to_csv('YOUR_DATA_FILE.csv', mode='a', header=False)
            # Note: while header = False, the index counting restarts at every new table appended.
            # So, use index counting to identify where to start editing a table
            i += 1
        except IndexError:
            break

tidy_saver(tables)